In [1]:
spark.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

In [23]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
data_path = os.path.join('..', 'class_16')

In [29]:
#movies = spark.read.csv(os.path.join(data_path, 'movies.csv'), header=True, inferSchema=True)
tags = spark.read.csv(os.path.join(data_path, 'tags.csv'), header=True, inferSchema=True)
ratings = spark.read.csv(os.path.join(data_path, 'ratings.csv'), header=True, inferSchema=True)

In [32]:
# 
s_tags = tags.sample(fraction=.1, seed=123454321)
s_tags.show()

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|    20|   4306|     Dreamworks|1459855607|
|    20|  96079|      nostalgic|1407930249|
|    79|   1057|         comedy|1483577176|
|    79|   1265|      feel-good|1483578282|
|    79|   1265|      time loop|1483578274|
|    79|   1784|      feel-good|1483577223|
|    79|   3481|          funny|1486673571|
|    79|   4823|    john cusack|1483577733|
|    79|   4878|    stereotypes|1487184022|
|    79|   4973| modern fantasy|1487184113|
|    79|   4973|       stylized|1487184057|
|    79|   7361|      dreamlike|1483576002|
|    79|   7361|    imagination|1483576038|
|    79|  44191|        torture|1487183453|
|    79|  60069|     love story|1483660138|
|    79|  68954|          funny|1483660119|
|    79|  69757|           love|1483576100|
|    79|  69757|meaning of love|1483576105|
|    79|  86882|    Owen Wilson|1483576198|
|    79|  86882|          witty|

In [42]:
from pyspark.sql.window import Window

wc = Window().orderBy(F.col('length_tag').asc())

(s_tags
 .select(F.col('tag'), F.length(F.col('tag')).alias('length_tag')) 
 .select('*', F.percent_rank().over(wc).alias('length_rank'))
 .filter(F.col('length_rank') > .9)
).show(20, truncate =False)

22/11/23 20:36:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:36:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:36:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:36:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:36:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------------------+----------+------------------+
|tag               |length_tag|length_rank       |
+------------------+----------+------------------+
|visually appealing|18

In [57]:
(s_tags.filter(F.col('userId') == 12825)).show()

+------+-------+--------------------+----------+
|userId|movieId|                 tag| timestamp|
+------+-------+--------------------+----------+
| 12825|    175|Nudity (Full Fron...|1436987982|
| 12825|   1676|     Robert Heinlein|1436988505|
| 12825|   5237|    military academy|1427144270|
| 12825|   7976|Nudity (Full Fron...|1438465726|
| 12825|   8361|    Post apocalyptic|1437190741|
| 12825|  31408|    Nudity (Topless)|1436988062|
| 12825|  44663|          Berlusconi|1378249251|
| 12825|  47997|                dumb|1460650564|
| 12825|  47997|            dystopia|1460589770|
| 12825|  50379|nudity (full fron...|1426540630|
| 12825|  59729|              incest|1426540684|
| 12825|  61240|       coming of age|1427504623|
| 12825|  76093|          friendship|1427143012|
| 12825|  92439|       coming of age|1427143713|
| 12825|  97913|               geeky|1427143206|
| 12825| 115713|artificial intell...|1438555078|
| 12825| 117322|              incest|1436988221|
+------+-------+----

In [55]:
(s_tags
 .groupBy(F.col('userId'))
 .agg(F.count('*').alias('user_rates'))
 .select('*', F.percent_rank().over(Window().orderBy(F.col('user_rates').asc())).alias('activity_ntile'))
 .filter(F.col('activity_ntile') > .9)
).show()

22/11/23 20:58:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:58:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:58:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:58:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:58:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 20:58:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 2

In [50]:
# 1. First Tag population

# -> tag.id user_id timestamp
#.     tag_1 user_1   t_1_1
#.     tag_1 user_2
#.     tag_1 user_...

#    -> s1, s2, s3, s4, ...
# 


'''
s_tags.
'''

wc = Window().partitionBy(F.col('userId')).orderBy(F.col('timestamp').asc())

(s_tags
 .select('*', F.rank().over(wc).alias('rank'))
 .filter(F.col('rank') == 1)
).show()

+------+-------+-----------------+----------+----+
|userId|movieId|              tag| timestamp|rank|
+------+-------+-----------------+----------+----+
|    20|  96079|        nostalgic|1407930249|   1|
|   111| 109487|      Masterpiece|1490443287|   1|
|   429| 116821|           satire|1415887689|   1|
|   471|    260|           sci-fi|1437883183|   1|
|   479|    593|    psychological|1430333622|   1|
|   519|  92259|       best actor|1433005248|   1|
|   540|   2268|   Jack Nicholson|1154193409|   1|
|   836|   6297|Adapted from book|1303360334|   1|
|   886|    260|  Science Fiction|1433535242|   1|
|   983|   2423|           funny!|1189035751|   1|
|   995|    260|       Bad acting|1436699043|   1|
|  1025|    296|          intense|1434153098|   1|
|  1145|  51080|     Chris Cooper|1187569588|   1|
|  1244|   1080|           parody|1272629168|   1|
|  1250|   1035|       true story|1223304741|   1|
|  1294|    260|   Groundbreaking|1441387838|   1|
|  1352|    926|     femme fata

In [49]:
ratings_s = ratings.sample(fraction=.01, seed=123454321)

In [9]:
ratings_s.count()

260095

In [14]:
movies.show(5)

22/11/23 20:07:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-----+-----------------+----------------------+--------+-----------------------+----------------------+----------------+----------------------+------------+--------------------+---------------+--------------------+---------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------+--------------+------+----------+----------------------+----------+------------+--------------------+
|color|    director_name|num_critic_for_reviews|duration|director_facebook_likes|actor_3_facebook_likes|    actor_2_name|actor_1_facebook_likes|       gross|              genres|   actor_1_name|         movie_title|num_voted_users|cast_total_facebook_likes|        actor_3_name|facenumber_in_poster|       plot_keywords|     movie_imdb_lin

In [15]:
# What's the largest time between rates from the same user to a given movie. 
#
import pyspark.sql.functions as F

p_var = 'percentage_users_rated_twice_same_director'
t_var = 'total_users_rated_twice_same_director'

(ratings_s
 .join(movies.select('movieId', 'director_name'), on='movieId', how='left')
 .groupBy(F.col('userId'), F.col('director_name'))
 .agg(F.count('*').alias('duplicates'))
 .select('*', (F.col('duplicates') > 1).cast('int').alias('is_duplicate'))
 .selectExpr(f'avg(is_duplicate) as {p_var}',f'sum(is_duplicate) as {t_var}')
).show()

AnalysisException: Column 'movieId' does not exist. Did you mean one of the following? [budget, color, country, genres, movie_title, gross, duration, language, imdb_score, movie_imdb_link, title_year, plot_keywords, actor_1_name, actor_2_name, actor_3_name, aspect_ratio, content_rating, director_name, num_voted_users, movie_facebook_likes, num_user_for_reviews, num_critic_for_reviews, facenumber_in_poster, actor_1_facebook_likes, actor_2_facebook_likes, actor_3_facebook_likes, director_facebook_likes, cast_total_facebook_likes];
'Project ['movieId, director_name#18]
+- Relation [color#17,director_name#18,num_critic_for_reviews#19,duration#20,director_facebook_likes#21,actor_3_facebook_likes#22,actor_2_name#23,actor_1_facebook_likes#24,gross#25,genres#26,actor_1_name#27,movie_title#28,num_voted_users#29,cast_total_facebook_likes#30,actor_3_name#31,facenumber_in_poster#32,plot_keywords#33,movie_imdb_link#34,num_user_for_reviews#35,language#36,country#37,content_rating#38,budget#39,title_